# 파생변수 생성 : 이전 주차의 순위

- 다음 순위를 예측할 때 가장 최근 주차의 순위가 영향을 크게 미칠 것으로 예상되어 이전 주차의 순위를 파생변수로 추가

In [1]:
import pandas as pd

## 데이터 로드

In [2]:
data = pd.read_csv('merged_circle_finish.csv', index_col=0)
data.index.name = None
data.head()

,date,Rank,Title,Artist_x,Production,Weekly Views,Genre,Runtime,New_Artist,n_score,g_score,ky_chart_on,Ratio
0,2023-09-24,1,Love Lee,AKMU (악뮤),YG Entertainment,11929992.0,가요 / 댄스,3:00,악뮤,94.32015,100.0,False,94.32015
1,2023-09-24,2,후라이의 꿈,AKMU (악뮤),YG Entertainment,11929992.0,가요 / 블루스/포크,3:25,악뮤,94.32015,100.0,False,94.32015
3,2023-09-24,4,Super Shy,NewJeans,ADOR,11655019.0,가요 / 댄스,2:35,뉴진스,10.55509,6.0,False,10.55509
4,2023-09-24,5,Seven (feat. Latto) - Clean Ver.,정국,BIGHIT MUSIC,4173726.0,가요 / 댄스,3:04,정국,61.50958,45.0,False,61.50958
5,2023-09-24,6,ETA,NewJeans,ADOR,11655019.0,가요 / 댄스,2:31,뉴진스,10.55509,6.0,False,10.55509


## 파생 변수 생성
> - 먼저 가장 최근 날짜부터 시작하여 일주일 전 데이터를 검색하여 해당 이전 주차 순위가 존재하면 해당 순위를, 없으면 결측치로 남겨놓음
> - 이 과정을 가장 오래된 날짜의 1주일 후까지 수행하고, 가장 오래된 날짜는 수동으로 입력할 예정

In [3]:
from datetime import datetime, timedelta

#### Previous_Rank 파생 변수 생성 코드

In [4]:
timestamp = datetime.strptime('2024-09-08', '%Y-%m-%d') # 시작 날짜
endtime = datetime.strptime('2023-10-01', '%Y-%m-%d') # 끊을 날짜

# 이전 주차 랭크를 선정할'Previous_Rank' 컬럼 생성
data['Previous_Rank'] = None

# 설정할 끊을 날짜에 도달하기까지 반복
while timestamp >= endtime:
    timestamp_str = timestamp.strftime('%Y-%m-%d') # 현재 날짜 문자열
    prev_timestamp_str = (timestamp - timedelta(days=7)).strftime('%Y-%m-%d') # 일주일 전 날짜 문자열
    
    data_now = data[data['date'] == timestamp_str] # 현재 날짜에 해당하는 랭킹 정보 데이터프레임 추출
    data_prev = data[data['date'] == prev_timestamp_str] # 일주일 전 날짜에 해당하는 랭킹 정보 데이터프레임 추출
    
    # 현재 날짜 데이터프레임의 각 행(노래)을 반복
    for row in data_now.itertuples():
        # 현재 노래의 일주일 전 랭크를 추출
        prev_rank = data_prev[data_prev['Title'] == row.Title]['Rank']
        
        # 만약 랭크가 추출되었다면
        if not prev_rank.empty:
            # 현재 노래의 현재 날짜 행에 일주일 전 랭크를 Previous_Rank 칼럼으로 저장
            data.loc[(data['date'] == timestamp_str) & (data['Title'] == row.Title), 'Previous_Rank'] = prev_rank.values[0]
        # 랭크가 추출되지 않았다면(새로 나온 곡이거나 일주일 전에 차트인되지 않은 곡이라면) Previous_Rank 칼럼은 결측치로 유지됨
    
    # 현재 날짜를 일주일 전으로 이동
    timestamp -= timedelta(days=7)

data.head()

,date,Rank,Title,Artist_x,Production,Weekly Views,Genre,Runtime,New_Artist,n_score,g_score,ky_chart_on,Ratio,Previous_Rank
0,2023-09-24,1,Love Lee,AKMU (악뮤),YG Entertainment,11929992.0,가요 / 댄스,3:00,악뮤,94.32015,100.0,False,94.32015,None
1,2023-09-24,2,후라이의 꿈,AKMU (악뮤),YG Entertainment,11929992.0,가요 / 블루스/포크,3:25,악뮤,94.32015,100.0,False,94.32015,None
3,2023-09-24,4,Super Shy,NewJeans,ADOR,11655019.0,가요 / 댄스,2:35,뉴진스,10.55509,6.0,False,10.55509,None
4,2023-09-24,5,Seven (feat. Latto) - Clean Ver.,정국,BIGHIT MUSIC,4173726.0,가요 / 댄스,3:04,정국,61.50958,45.0,False,61.50958,None
5,2023-09-24,6,ETA,NewJeans,ADOR,11655019.0,가요 / 댄스,2:31,뉴진스,10.55509,6.0,False,10.55509,None


## 파생변수 결측치 처리
- Previous_Rank에 결측치가 존재한다는 것은 해당 곡이 신곡이거나 순위권 밖에서 올라온 곡이라는 뜻
- 따라서 이러한 결측치는 201위로 처리하되, 이전 차트에 존재하지 않았다는 의미의 New_Rank 변수를 별도로 생성
- New_Rank가 True이면 일주일 전 차트에 없었던 곡이라는 의미

### New_Rank 파생변수 생성

- Previous_Rank가 결측치라면 True 값이 입력되고 결측치가 아니라면 False 값이 입력됨

In [5]:
data['New_Rank'] = data['Previous_Rank'].isnull()
data.head()

,date,Rank,Title,Artist_x,Production,Weekly Views,Genre,Runtime,New_Artist,n_score,g_score,ky_chart_on,Ratio,Previous_Rank,New_Rank
0,2023-09-24,1,Love Lee,AKMU (악뮤),YG Entertainment,11929992.0,가요 / 댄스,3:00,악뮤,94.32015,100.0,False,94.32015,None,True
1,2023-09-24,2,후라이의 꿈,AKMU (악뮤),YG Entertainment,11929992.0,가요 / 블루스/포크,3:25,악뮤,94.32015,100.0,False,94.32015,None,True
3,2023-09-24,4,Super Shy,NewJeans,ADOR,11655019.0,가요 / 댄스,2:35,뉴진스,10.55509,6.0,False,10.55509,None,True
4,2023-09-24,5,Seven (feat. Latto) - Clean Ver.,정국,BIGHIT MUSIC,4173726.0,가요 / 댄스,3:04,정국,61.50958,45.0,False,61.50958,None,True
5,2023-09-24,6,ETA,NewJeans,ADOR,11655019.0,가요 / 댄스,2:31,뉴진스,10.55509,6.0,False,10.55509,None,True


In [6]:
import numpy as np

- Previous_Rank 결측치를 전부 201위로 변경

In [7]:
data['Previous_Rank'] = data['Previous_Rank'].fillna(201)
data.head()

C:\Users\hyeon\AppData\Local\Temp\ipykernel_27092\2824885555.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Previous_Rank'] = data['Previous_Rank'].fillna(201)


,date,Rank,Title,Artist_x,Production,Weekly Views,Genre,Runtime,New_Artist,n_score,g_score,ky_chart_on,Ratio,Previous_Rank,New_Rank
0,2023-09-24,1,Love Lee,AKMU (악뮤),YG Entertainment,11929992.0,가요 / 댄스,3:00,악뮤,94.32015,100.0,False,94.32015,201,True
1,2023-09-24,2,후라이의 꿈,AKMU (악뮤),YG Entertainment,11929992.0,가요 / 블루스/포크,3:25,악뮤,94.32015,100.0,False,94.32015,201,True
3,2023-09-24,4,Super Shy,NewJeans,ADOR,11655019.0,가요 / 댄스,2:35,뉴진스,10.55509,6.0,False,10.55509,201,True
4,2023-09-24,5,Seven (feat. Latto) - Clean Ver.,정국,BIGHIT MUSIC,4173726.0,가요 / 댄스,3:04,정국,61.50958,45.0,False,61.50958,201,True
5,2023-09-24,6,ETA,NewJeans,ADOR,11655019.0,가요 / 댄스,2:31,뉴진스,10.55509,6.0,False,10.55509,201,True


- 남은 결측치가 존재하는지 확인 : 모두 0

In [8]:
data.isnull().sum()

date             0
Rank             0
Title            0
Artist_x         0
Production       0
Weekly Views     0
Genre            0
Runtime          0
New_Artist       0
n_score          0
g_score          0
ky_chart_on      0
Ratio            0
Previous_Rank    0
New_Rank         0
dtype: int64

CSV 파일로 저장

In [9]:
data.to_csv('circle_previous_rank.csv')

### 이후 작업
- 가장 오래된 주차는 수동으로 Previous_Rank를 직접 입력
- 해당 작업이 완료된 CSV 파일은 circle_previous_rank_2.csv로 저장